In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.prompts import PromptTemplate
import pickle


In [2]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./multivector_chroma_db_001"
)

with open("parent_documents.pkl", "rb") as f:
    parent_documents = pickle.load(f)


# 🔹 Reconstruís el store
store = InMemoryStore()
store.mset([(d.metadata["id"], d) for d in parent_documents])

# 🔹 Reconstruís el retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key="parent_id",
    search_kwargs={"k": 3} # reminder: top 3 similitud para los chunks resumidos, no para los fallos completos
)

/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_3731/244047182.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_3731/244047182.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


### Only answer generation

In [ ]:
# Define LLM

llm = ChatOpenAI(model='o4-mini')

prompt_template = PromptTemplate.from_template("""
Sos un asistente jurídico.
Usá exclusivamente el contenido de los documentos proporcionados para responder la consulta.
Si encontrás documentos relacionados, proporcioná la información correspondiente de dichos documentos.
No inventes ni infieras información que no esté presente en los textos.
Si no hay jurisprudencia relevante, indicá claramente que no la encontrás.
Tampoco menciones ningún documento si no encontraste juridisprudencia relevante.

Documentos:
{context}

Pregunta:
{question}

Respuesta:
""")

# Cadena que usa esos campos
combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_template)

from langchain_core.runnables import RunnableLambda

def print_metadata(x):
    print("\n[Metadata de los documentos relevantes]:")
    for i, doc in enumerate(x["context"]):
        print(f"Documento {i+1}: {doc.metadata}")
    return x


# Este paso une todo: retriever + documents + fields necesarios
retrieval_chain = (
    RunnablePassthrough.assign(
        question=lambda x: x["input"],
        context=lambda x: retriever.invoke(x["input"])
    )
    | RunnableLambda(print_metadata)
    | combine_docs_chain
)

response = retrieval_chain.invoke({"input": "¿Existe algún fallo que trate la inconstitucionalidad de la ley de prenda?"})

print(response)

Sí. En autos “FCA Compañía Financiera S.A. c/ Yglesias Rodrigues s/ secuestro prendario” (Cámara Comercial, Sala F, Expte. COM 247/2024, sentencia 1/3/2024) se abordó expresamente la inconstitucionalidad de la Ley de Prenda con Registro (Ley 12.962), en particular de su art. 39, por resultar incompatible con:  
  • El art. 42 de la Constitución Nacional (protección del consumidor).  
  • Los arts. 3 y 37 de la Ley 24.240 (Ley de Defensa del Consumidor).  
  • El art. 1094 del Código Civil y Comercial.  

Ese tribunal sostuvo que, en materia de relaciones de consumo, el régimen de secuestro directo sin audiencia previa del deudor previsto en el art. 39 “importa una renuncia o restricción de derechos del consumidor” y, por tanto, debe reputarse “no convenido” y dispensarse por ser contrario al orden constitucional. La Sala F se apoyó, además, en el precedente de la Corte Suprema (“HSBC Bank Argentina S.A. c/ Martínez, Ramón Vicente s/ secuestro prendario”, Fallos: 342:1004), que había se

### Answer generation + Retrieve documents

In [ ]:
# Define LLM
llm = ChatOpenAI(model='o4-mini')

prompt_template = PromptTemplate.from_template("""
Sos un asistente jurídico.
Usá exclusivamente el contenido de los documentos proporcionados para responder la consulta.
Si encontrás documentos relacionados, proporcioná la información correspondiente de dichos documentos.
No inventes ni infieras información que no esté presente en los textos.
Si no hay jurisprudencia relevante, indicá claramente que no la encontrás.
Tampoco menciones ningún documento si no encontraste juridisprudencia relevante.

Documentos:
{context}

Pregunta:
{question}

Respuesta:
""")

combine_docs_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_template)

input_text = "¿Existe algún fallo que trate la inconstitucionalidad de la ley de prenda?"

# Step 1: retrieve relevant documents
retrieved_docs = retriever.invoke(input_text)

# Step 2: chain to analyze retrieved documents with the question
response = combine_docs_chain.invoke({
    "question": input_text,
    "context": retrieved_docs
})

# Model response
print("\n🧠 Respuesta del modelo:")
print(response)

# Metadata
print("\n📄 Documentos relevantes encontrados:")
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Documento {i+1} ---")
    print("Tribunal:", doc.metadata['Tribunal'])
    print("Sala:", doc.metadata['Sala'])
    print("Expediente:", doc.metadata['Expediente'])
    print("Caratula:", doc.metadata['Caratula'])
    print("Fecha de Sentencia:", doc.metadata['FechaSentencia'])



📄 Documentos relevantes encontrados:

--- Documento 1 ---
Tribunal: CAMARA COMERCIAL - SALA C
Sala: SALA C
Expediente: COM 014267/2024/CA002
Caratula: BASSI, GRACIELA NORA c/ SOCIEDAD ITALIANA DE BENEFICENCIA EN BUENOS AIRES s/AMPARO
Fecha de Sentencia: 08/04/2025

--- Documento 2 ---
Tribunal: CAMARA COMERCIAL - SALA F
Sala: SALA F
Expediente: COM 000247/2024/CA001
Caratula: FCA COMPAÑIA FINANCIERA S.A. c/ YGLESIAS RODRIGUES, CARLOS RUBEM s/SECUESTRO PRENDARIO
Fecha de Sentencia: 01/03/2024

--- Documento 3 ---
Tribunal: CAMARA COMERCIAL - SALA A
Sala: SALA A
Expediente: COM 015450/2021/22/CA037 - CA036
Caratula: CURTIEMBRES FONSECA SA s/ CONCURSO PREVENTIVO s/INCIDENTE DE REVISION DE CREDITO POR ERCROS S.A.
Fecha de Sentencia: 16/04/2024

🧠 Respuesta del modelo:
Sí. En la causa “FCA Compañía Financiera S.A. c/Yglesias Rodrigues s/secuestro prendario” (Expte. COM 247/2024, Cámara Comercial, Sala F, sentencia de 1.3.2024) se sostuvo que el art. 39 de la Ley de Prenda (D.L. 15.348/46) 